<a href="https://colab.research.google.com/github/ykitaguchi77/statistics_for_articles/blob/main/Canalicular_excision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import codecs
from scipy import stats
import numpy as np

In [3]:
csv_path = "/content/drive/MyDrive/作成中の論文/涙小管切断/涙小管切除症例.csv"

#Open csv
#with codecs.open(csv_path, "r", "Shift-JIS", "ignore") as file:
with codecs.open(csv_path, "r", "UTF-8", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=0)

#**フォロー期間12ヶ月以上を抜き出し**

In [21]:
df_long = df[df['Observation period (month)'] >= 12]
len(df_long)
#df_long.columns
df_long

,ID,Patient NO,name,OP date,OP date2,Age (yrs),M/F,Background disease,History,Side,Observation period (month),Punctum_upper,Punctum_lower,Pre Schirmer I,TMR_pre,TMR_post,FBUT_Pre,FBUT_post,Fluo_A_pre,Fluo_D_pre,Fluo_A_post,Fluo_D_post,Filamentosa_pre,Filamentosa_post,Post tear meniscus,Recurrence,Recurrent period (month),Unnamed: 27,Mibum finding,Mubum score,pre eyedrop,post eyedrop,備,Backgroud_Sjogren,Background_GVHD,Gender_F
0,8889238,1,ｺ,20180920.0,20181130.0,43.0,F,"Sjogren, SLE, RA",Punctal occulusion,R,36.0,Second,First,1.0,NaN,NaN,2.0,7.0,3.0,2.0,1.0,1.0,0.0,0.0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,斜視術,1.0,0.0,1.0
1,8889238,1,ｺ,20181227.0,20190815.0,43.0,F,"Sjogren, SLE, RA",Punctal occulusion,L,36.0,Second,First,3.0,NaN,NaN,2.0,7.0,2.0,2.0,0.0,0.0,0.0,0.0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0
2,7971399,2,ﾐｱ,20181227.0,NaN,74.0,F,Sjogren,Punctal occulusion,R,30.0,First,First,2.0,NaN,NaN,NaN,NaN,2.0,3.0,0.0,0.0,0.0,0.0,NaN,Upper,8.0,NaN,NaN,NaN,NaN,NaN,"Involutional entropion concurrent surgery, cor...",1.0,0.0,1.0
5,10546394,4,ﾊｼｼ,20190516.0,20200513.0,67.0,F,GVHD,NaN,R,17.0,First,First,1.0,NaN,NaN,2.0,NaN,1.0,2.0,1.0,2.0,0.0,0.0,NaN,Upper,5.0,NaN,黉刳,NaN,M,SS+TvO+Eso,Cytomegaro retinitis PPVtriple,0.0,1.0,1.0
6,10546394,4,ﾊｼｼ,20190516.0,NaN,67.0,F,GVHD,NaN,L,29.0,First,First,1.0,NaN,NaN,2.0,NaN,2.0,2.0,1.0,2.0,0.0,0.0,NaN,None,NaN,NaN,黉刳,NaN,M,SS+TvO,NaN,0.0,1.0,1.0
7,10696292,5,ﾂｲ,20190604.0,NaN,79.0,F,Sjogren,Punctal occulusion,R,19.0,First,First,2.0,NaN,NaN,1.0,NaN,3.0,2.0,0.0,0.0,0.0,0.0,NaN,None,NaN,NaN,NaN,NaN,D,M,NaN,1.0,0.0,1.0
8,10749858,6,ﾏﾾﾋｻ,20190611.0,NaN,79.0,F,Sjogren,NaN,R,31.0,First,First,3.0,NaN,NaN,1.0,2.0,3.0,3.0,3.0,1.0,0.0,0.0,NaN,None,NaN,NaN,NaN,NaN,M+D+SS,D+SS,NaN,1.0,0.0,1.0
9,10749858,6,ﾏﾾﾋｻ,20190611.0,NaN,79.0,F,Sjogren,Punctal occulusion,L,27.0,First,First,2.0,NaN,NaN,1.0,2.0,2.0,3.0,2.0,1.0,0.0,0.0,NaN,None,NaN,NaN,NaN,NaN,M+D+SS,D+SS,NaN,1.0,0.0,1.0
10,6720655,7,ﾗﾐﾖｺ,20190611.0,NaN,64.0,F,RA+ドライアイ、角膜感染,NaN,L,32.0,First,First,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,1.0,1.0,0.0,NaN,None,NaN,NaN,NaN,NaN,D+TvO+Rd+CV+SS,D+TvO+Rd+CV+SS,NaN,0.0,0.0,1.0
11,11303238,8,ｾ,20190704.0,NaN,50.0,F,Sjogren,NaN,R,28.0,First,None,4.0,NaN,NaN,NaN,NaN,2.0,3.0,2.0,2.0,0.0,0.0,NaN,None,NaN,NaN,NaN,NaN,D+M,D+M,上適応あるが希望な,1.0,0.0,1.0


In [ ]:
from sklearn.preprocessing import OneHotEncoder

#解析のため文字列を数値に変える
#症例数が少なければ、csvの列を作り直した方が早い

#男性をゼロ、女性を1に
df_long_1 = df_long.replace({"M/F": {"M": 0, "F": 1}})

df_long_1["Sjogren"] = 0
df_long_1["GVHD"] = 0

#pd.get_dummiesを用いてone-hot encodingを行う
df_long_1 = pd.get_dummies(df_long, columns=["M/F"])
df_long_2 = pd.get_dummies(df_long, columns=["Background disease"])
df_long_2

#Scikit learnからone hot encoderを用いる手法
encoder = OneHotEncoder(sparse=False)

arr = encoder.fit_transform(df_long["M/F"].values.reshape(-1, 1))
df_MF = pd.DataFrame(arr)
df_MF.columns = encoder.get_feature_names_out(["gender"])
df_MF

arr = encoder.fit_transform(df_long["Background disease"].values.reshape(-1, 1))
df_BD = pd.DataFrame(arr)
df_BD.columns = encoder.get_feature_names_out(["Background"])
df_BD

In [ ]:
df_long.value_counts("Side")

In [6]:
df_long.value_counts("History")

History
Punctal occulusion    5
dtype: int64

In [7]:
df_long.describe()["Pre Schirmer I"]

count    25.000000
mean      3.000000
std       2.327373
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       8.000000
Name: Pre Schirmer I, dtype: float64

In [ ]:
df_long.value_counts("History")

History
Punctal occulusion    5
dtype: int64

In [ ]:
print(df_long.describe()["Fluo_D_pre"])
print(df_long.value_counts("Fluo_D_pre"))

count    26.000000
mean      2.500000
std       0.583095
min       1.000000
25%       2.000000
50%       3.000000
75%       3.000000
max       3.000000
Name: Fluo_D_pre, dtype: float64
Fluo_D_pre
3.0    14
2.0    11
1.0     1
dtype: int64


In [8]:
print(df_long.describe()["Fluo_D_post"])
print(df_long.value_counts("Fluo_D_post"))

count    26.000000
mean      1.423077
std       0.986836
min       0.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       3.000000
Name: Fluo_D_post, dtype: float64
Fluo_D_post
2.0    11
0.0     6
1.0     6
3.0     3
dtype: int64


In [9]:
print(df_long.describe()["Filamentosa_pre"])
print(df_long.value_counts("Filamentosa_pre"))

count    26.000000
mean      0.192308
std       0.401918
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
Name: Filamentosa_pre, dtype: float64
Filamentosa_pre
0.0    21
1.0     5
dtype: int64


In [10]:
print(df_long.describe()["Filamentosa_post"])
print(df_long.value_counts("Filamentosa_post"))

count    26.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: Filamentosa_post, dtype: float64
Filamentosa_post
0.0    26
dtype: int64


In [24]:
print(df_long.describe()["Pre Schirmer I"])
print(df_long.value_counts("Pre Schirmer I"))


count    25.000000
mean      3.000000
std       2.327373
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       8.000000
Name: Pre Schirmer I, dtype: float64
Pre Schirmer I
2.0    7
1.0    6
3.0    5
7.0    4
0.0    1
4.0    1
8.0    1
dtype: int64


In [25]:
print(df_long.describe()["Recurrent period (month)"])


count    5.000000
mean     3.800000
std      2.774887
min      1.000000
25%      2.000000
50%      3.000000
75%      5.000000
max      8.000000
Name: Recurrent period (month), dtype: float64


In [26]:
print(df_long.columns)

Index(['ID', 'Patient NO', 'name', 'OP date', 'OP date2', 'Age (yrs)', 'M/F',
       'Background disease', 'History', 'Side', 'Observation period (month)',
       'Punctum_upper', 'Punctum_lower', 'Pre Schirmer I', 'TMR_pre',
       'TMR_post', 'FBUT_Pre', 'FBUT_post', 'Fluo_A_pre', 'Fluo_D_pre',
       'Fluo_A_post', 'Fluo_A_post.1', 'Filamentosa_pre', 'Filamentosa_post',
       'Post tear meniscus', 'Recurrence', 'Recurrent period (month)',
       'Unnamed: 27', 'Mibum finding', 'Mubum score', 'pre eyedrop',
       'post eyedrop', '備'],
      dtype='object')


In [5]:
#Fluoの変化は有意
stats.wilcoxon(df_long['Fluo_D_pre'], df_long['Fluo_D_post'])

WilcoxonResult(statistic=4.5, pvalue=0.00031767360182023084)

In [6]:
print(df_long.columns)

Index(['ID', 'Patient NO', 'name', 'OP date', 'OP date2', 'Age (yrs)', 'M/F',
       'Background disease', 'History', 'Side', 'Observation period (month)',
       'Punctum_upper', 'Punctum_lower', 'Pre Schirmer I', 'TMR_pre',
       'TMR_post', 'FBUT_Pre', 'FBUT_post', 'Fluo_A_pre', 'Fluo_D_pre',
       'Fluo_A_post', 'Fluo_D_post', 'Filamentosa_pre', 'Filamentosa_post',
       'Post tear meniscus', 'Recurrence', 'Recurrent period (month)',
       'Unnamed: 27', 'Mibum finding', 'Mubum score', 'pre eyedrop',
       'post eyedrop', '備', 'Backgroud_Sjogren', 'Background_GVHD',
       'Gender_F'],
      dtype='object')


In [25]:
#欠損値の確認
df_long.isnull().sum()

ID                             0
Patient NO                     0
name                           0
OP date                        0
OP date2                      23
Age (yrs)                      0
M/F                            0
Background disease             0
History                       21
Side                           0
Observation period (month)     0
Punctum_upper                  0
Punctum_lower                  0
Pre Schirmer I                 0
TMR_pre                       26
TMR_post                      26
FBUT_Pre                      19
FBUT_post                     22
Fluo_A_pre                     0
Fluo_D_pre                     0
Fluo_A_post                    0
Fluo_D_post                    0
Filamentosa_pre                0
Filamentosa_post               0
Post tear meniscus            24
Recurrence                     0
Recurrent period (month)      21
Unnamed: 27                   26
Mibum finding                 24
Mubum score                   26
pre eyedro

In [26]:
#欠損値を中央値で埋める
df_long['Pre Schirmer I'] = df_long['Pre Schirmer I'].fillna(df_long['Pre Schirmer I'].median())

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [66]:
from sklearn import linear_model

#X = df_long.loc[:, ["Fluo_D_pre", "Filamentosa_pre", "Pre Schirmer I", "Age (yrs)", "Gender_F", "Backgroud_Sjogren", "Background_GVHD"]].values
X = df_long.loc[:, ["Fluo_D_pre", "Filamentosa_pre", "Pre Schirmer I"]].values

Y = df_long["Fluo_D_post"].values

model = linear_model.LinearRegression()
model.fit(X,Y)
model.coef_

# データフレームに変換し、カラム名とインデックス名を指定
#df_coefficient = pd.DataFrame(model.coef_, columns=["Fluo_D_post"], index=["Fluo_D_pre", "Filamentosa_pre", "Pre Schirmer I", "Age (yrs)", "Gender_F", "Backgroud_Sjogren", "Background_GVHD"])
df_coefficient = pd.DataFrame(model.coef_, columns=["Fluo_D_post"], index=["Fluo_D_pre", "Filamentosa_pre", "Pre Schirmer I"])
df_coefficient

,Fluo_D_post
Fluo_D_pre,0.334116
Filamentosa_pre,-0.817577
Pre Schirmer I,0.003719


In [67]:
import statsmodels.api as sm

model = sm.OLS(Y, sm.add_constant(X))
results = model.fit()
print(results.summary())
# --> 術後SPKの重症度と有意に相関する項目なし


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     1.187
Date:                Mon, 28 Feb 2022   Prob (F-statistic):              0.338
Time:                        05:01:51   Log-Likelihood:                -34.088
No. Observations:                  26   AIC:                             76.18
Df Residuals:                      22   BIC:                             81.21
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7340      0.899      0.817      0.4

In [36]:
df_long.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))


,OP date,OP date2,Age (yrs),Observation period (month),Pre Schirmer I,TMR_pre,TMR_post,FBUT_Pre,FBUT_post,Fluo_A_pre,Fluo_D_pre,Fluo_A_post,Fluo_D_post,Filamentosa_pre,Filamentosa_post,Recurrent period (month),Unnamed: 27,Mubum score,Backgroud_Sjogren,Background_GVHD,Gender_F
count,26,3,26,26,26,0,0,7,4,26,26,26,26,26,26,5,0,0,26,26,26
mean,2.01964e+07,2.01908e+07,59,22.6154,2.96154,nan,nan,1.57143,4.5,2.11538,2.5,1.42308,1.42308,0.192308,0,3.8,nan,nan,0.5,0.346154,0.730769
std,9610.56,9691.5,14.7296,7.98537,2.28877,nan,nan,0.534522,2.88675,0.711445,0.583095,1.02657,0.986836,0.401918,0,2.77489,nan,nan,0.509902,0.485165,0.452344
min,2.01809e+07,2.01811e+07,37,12,0,nan,nan,1,2,1,1,0,0,0,0,1,nan,nan,0,0,0
25%,2.01906e+07,2.0186e+07,49,15.25,1.25,nan,nan,1,2,2,2,1,1,0,0,2,nan,nan,0,0,0.25
50%,2.0191e+07,2.01908e+07,55,24,2,nan,nan,2,4.5,2,3,1.5,2,0,0,3,nan,nan,0.5,0,1
75%,2.0201e+07,2.01957e+07,71.5,28.75,3,nan,nan,2,7,3,3,2,2,0,0,5,nan,nan,1,1,1
max,2.02104e+07,2.02005e+07,86,36,8,nan,nan,2,7,3,3,3,3,1,0,8,nan,nan,1,1,1


In [ ]:
# matplotlibを日本語表示に対応させるモジュール
import japanize_matplotlib

# pandasのplottingメソッドをインポート
from pandas import plotting

# 散布図行列を表示
df_Z = df_long[["Fluo_D_pre", "Fluo_D_post", "Filamentosa_pre", "Pre Schirmer I", "Age (yrs)", "Gender_F", "Backgroud_Sjogren", "Background_GVHD"]]
plotting.scatter_matrix(df_Z, figsize=(20, 20), alpha=0.8)

In [44]:
df_Z.corr()
# --> 相関が多少でもありそうなのは、pre-ShirmerとFluo-D-preぐらい

,Fluo_D_pre,Fluo_D_post,Filamentosa_pre,Pre Schirmer I,Age (yrs),Gender_F,Backgroud_Sjogren,Background_GVHD
Fluo_D_pre,1.000000,0.173786,0.085340,0.554486,0.083831,-0.227480,0.067267,-0.070697
Fluo_D_post,0.173786,1.000000,-0.314187,0.042912,-0.030270,-0.003446,-0.357718,0.183159
Filamentosa_pre,0.085340,-0.314187,1.000000,0.225777,0.060810,-0.143856,0.097590,-0.149904
Pre Schirmer I,0.554486,0.042912,0.225777,1.000000,0.086615,-0.628573,-0.051412,-0.167641
Age (yrs),0.083831,-0.030270,0.060810,0.086615,1.000000,0.156090,0.292918,-0.391813
Gender_F,-0.227480,-0.003446,-0.143856,-0.628573,0.156090,1.000000,0.433555,-0.287417
Backgroud_Sjogren,0.067267,-0.357718,0.097590,-0.051412,0.292918,0.433555,1.000000,-0.727607
Background_GVHD,-0.070697,0.183159,-0.149904,-0.167641,-0.391813,-0.287417,-0.727607,1.000000


In [61]:
from scipy import stats
df_Kruskal = df_long[["Fluo_D_pre", "Pre Schirmer I"]]
A = df_Kruskal.query("Fluo_D_pre == 1")
B = df_Kruskal.query("Fluo_D_pre == 2")
C = df_Kruskal.query("Fluo_D_pre == 3")
print("Pre_Fluo D1:")
print(A.describe())
print("Pre_Fluo D2:")
print(B.describe())
print("Pre_Fluo D3:")
print(C.describe())


x = A["Pre Schirmer I"]
y = B["Pre Schirmer I"]
z = C["Pre Schirmer I"]

print("Pre_Fluo D1:")
print(x)
print("Pre_Fluo D2:")
print(y)
print("Pre_Fluo D3:")
print(z)


stats.kruskal(x, y, z)

#--> シルマー値が低いほど、術前の角膜上皮障害が強い

Pre_Fluo D1:
       Fluo_D_pre  Pre Schirmer I
count         1.0             1.0
mean          1.0             1.0
std           NaN             NaN
min           1.0             1.0
25%           1.0             1.0
50%           1.0             1.0
75%           1.0             1.0
max           1.0             1.0
Pre_Fluo D2:
       Fluo_D_pre  Pre Schirmer I
count        11.0       11.000000
mean          2.0        1.636364
std           0.0        0.924416
min           2.0        0.000000
25%           2.0        1.000000
50%           2.0        2.000000
75%           2.0        2.000000
max           2.0        3.000000
Pre_Fluo D3:
       Fluo_D_pre  Pre Schirmer I
count        14.0       14.000000
mean          3.0        4.142857
std           0.0        2.476261
min           3.0        1.000000
25%           3.0        2.000000
50%           3.0        3.000000
75%           3.0        7.000000
max           3.0        8.000000
Pre_Fluo D1:
24    1.0
Name: Pre Schirmer I

KruskalResult(statistic=9.260293470938283, pvalue=0.009753327838200834)

In [63]:
from scipy import stats
df_Kruskal = df_long[["Fluo_D_post", "Pre Schirmer I"]]
A = df_Kruskal.query("Fluo_D_post == 1")
B = df_Kruskal.query("Fluo_D_post == 2")
C = df_Kruskal.query("Fluo_D_post == 3")
print("Post_Fluo D1:")
print(A.describe())
print("Post_Fluo D2:")
print(B.describe())
print("Post_Fluo D3:")
print(C.describe())


x = A["Pre Schirmer I"]
y = B["Pre Schirmer I"]
z = C["Pre Schirmer I"]

print("Post_Fluo D1:")
print(x)
print("Post_Fluo D2:")
print(y)
print("Post_Fluo D3:")
print(z)


stats.kruskal(x, y, z)

#--> シルマー値が低いほど、術前の角膜上皮障害が強い

Post_Fluo D1:
       Fluo_D_post  Pre Schirmer I
count          6.0        6.000000
mean           1.0        2.833333
std            0.0        2.136976
min            1.0        1.000000
25%            1.0        2.000000
50%            1.0        2.000000
75%            1.0        2.750000
max            1.0        7.000000
Post_Fluo D2:
       Fluo_D_post  Pre Schirmer I
count         11.0       11.000000
mean           2.0        2.545455
std            0.0        2.659460
min            2.0        0.000000
25%            2.0        1.000000
50%            2.0        1.000000
75%            2.0        3.000000
max            2.0        8.000000
Post_Fluo D3:
       Fluo_D_post  Pre Schirmer I
count          3.0        3.000000
mean           3.0        4.333333
std            0.0        2.309401
min            3.0        3.000000
25%            3.0        3.000000
50%            3.0        3.000000
75%            3.0        5.000000
max            3.0        7.000000
Post_Fluo D1:

KruskalResult(statistic=3.0465846308387783, pvalue=0.2179930024809657)

#**患者別**

In [ ]:
df_long_pts = df_long.drop_duplicates(subset=['Patient NO'], keep="last")
len(df_long_pts)
df_long_pts


In [ ]:
df_long_pts.describe()["Age (yrs)"]

In [ ]:
print(df_long_pts.value_counts("Background disease"))

Background disease
Sjogren             7
GVHD                5
BSK                 1
Post RT             1
RA+ドライアイ、角膜感染       1
Sjogren, SLE, RA    1
Sjogren未            1
強皮                  1
dtype: int64


In [ ]:
"""

M/F
F    13
M     5
dtype: int64


Background disease
Sjogren             6
GVHD                5
BSK                 1
Post RT             1
RA+ドライアイ、角膜感染       1
Shogren             1
Shogren, SLE, RA    1
Shogren未検           1
強皮症                 1
dtype: int64


History
Punctal occulusion    4
dtype: int64


Side
L    12
R     6
dtype: int64


Observation period (month)
28.0    3
12.0    2
29.0    2
13.0    1
14.0    1
15.0    1
16.0    1
19.0    1
23.0    1
25.0    1
27.0    1
30.0    1
32.0    1
36.0    1
dtype: int64
"""